In [1]:
from dataclasses import dataclass
import numpy as np
import numba as nb
from numba import njit, prange
from tqdm.auto import tqdm
from numba.typed import List, Dict
from scipy.special import factorial
from prettytable import PrettyTable 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.Util import dt_factorial, _dt_factorial, dt_sum, _sort_and_fill, \
    sort_and_fill, binomial_coefficient, _fill_zeros, fill_zeros, _remove_tail, \
    remove_tail
from src.DerivativeTypes import generate_derivative_types, _generate_derivative_types
from src.Combinatorics import _compute_etas, compute_etas, _compute_zetas, compute_zetas, \
    _compute_sorted_zetas, compute_sorted_zetas, _number_of_representations, number_of_representations
from src.Hashing import _der_type_to_hash, der_type_to_hash, _der_types_to_hashes, der_types_to_hashes
from src.DerivativeBounds import _make_dbound_dict, make_dbound_dict

## Generate Derivative Types

In [7]:
generate_derivative_types(4, 6)

ListType[array(int16, 1d, A)]([[1 1 1 1 0 0], [2 1 1 0 0 0], [2 2 0 0 0 0], [3 1 0 0 0 0], [4 0 0 0 0 0], ...])

In [8]:
[(list(l), sum(l)) for l in generate_derivative_types(4, 6)]

[([1, 1, 1, 1, 0, 0], 4),
 ([2, 1, 1, 0, 0, 0], 4),
 ([2, 2, 0, 0, 0, 0], 4),
 ([3, 1, 0, 0, 0, 0], 4),
 ([4, 0, 0, 0, 0, 0], 4)]

In [9]:
%%timeit
_generate_derivative_types(10, 500)

6.91 ms ± 692 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
generate_derivative_types(10, 500)

131 µs ± 4.72 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [11]:
%%time
ns = [1, 2, 3, 5, 10, 20] # , 50
ks = [1, 2, 3, 5, 10, 25, 50, 100, 250, 500, 1000]
table = PrettyTable(["k\\n"] + [str(n) for n in ns])
for k in ks:
    table.add_row([str(k)] + [f"{len(generate_derivative_types(n, k)):,}" for n in ns])
print(f'Sorted multi-indices with k entries summing up to n:')
print(table)

Sorted multi-indices with k entries summing up to n:
+------+---+---+---+---+----+-----+
| k\n  | 1 | 2 | 3 | 5 | 10 |  20 |
+------+---+---+---+---+----+-----+
|  1   | 1 | 1 | 1 | 1 | 1  |  1  |
|  2   | 1 | 2 | 2 | 3 | 6  |  11 |
|  3   | 1 | 2 | 3 | 5 | 14 |  44 |
|  5   | 1 | 2 | 3 | 7 | 30 | 192 |
|  10  | 1 | 2 | 3 | 7 | 42 | 530 |
|  25  | 1 | 2 | 3 | 7 | 42 | 627 |
|  50  | 1 | 2 | 3 | 7 | 42 | 627 |
| 100  | 1 | 2 | 3 | 7 | 42 | 627 |
| 250  | 1 | 2 | 3 | 7 | 42 | 627 |
| 500  | 1 | 2 | 3 | 7 | 42 | 627 |
| 1000 | 1 | 2 | 3 | 7 | 42 | 627 |
+------+---+---+---+---+----+-----+
CPU times: user 31.7 ms, sys: 2.67 ms, total: 34.3 ms
Wall time: 33.5 ms


## Derivative Type Hashing

In [12]:
n = 4
k = 6
[(arr, der_type_to_hash(arr, n, k)) for arr in generate_derivative_types(n, k)]

[(array([1, 1, 1, 1, 0, 0], dtype=int16), 85),
 (array([2, 1, 1, 0, 0, 0], dtype=int16), 22),
 (array([2, 2, 0, 0, 0, 0], dtype=int16), 10),
 (array([3, 1, 0, 0, 0, 0], dtype=int16), 7),
 (array([4, 0, 0, 0, 0, 0], dtype=int16), 4)]

In [13]:
n=25; k=50
hashes = der_types_to_hashes(generate_derivative_types(n, k), n, k)
print(len(hashes), len(set(hashes)))

1958 1958


In [14]:
%%timeit
[_der_type_to_hash(arr, n, k) for arr in generate_derivative_types(n, k)]

76 ms ± 3.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%%timeit
_der_types_to_hashes(generate_derivative_types(n, k), n, k)

17.9 ms ± 763 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%%timeit
der_types_to_hashes(generate_derivative_types(n, k), n, k)

13 ms ± 105 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Accessing Derivative Bounds

In [21]:
n = 4; k = 6
make_dbound_dict(der_types_to_hashes(generate_derivative_types(n, k), n, k), np.arange(5))

DictType[int64,float64]<iv=None>({85: 0.0, 22: 1.0, 10: 2.0, 7: 3.0, 4: 4.0})

In [22]:
%%timeit
n=25; k=50
_make_dbound_dict(der_types_to_hashes(generate_derivative_types(n, k), n, k), np.arange(1958))

17 ms ± 752 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
%%timeit
n=25; k=50
make_dbound_dict(der_types_to_hashes(generate_derivative_types(n, k), n, k), np.arange(1958))

14.2 ms ± 1.01 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Computing the Derivative Bound

## Compute the Cummulated Bounds of $g$

### Compute $\eta$ , $\zeta$
#### Helper Functions

In [24]:
arr = np.zeros(100, dtype=np.int16)
arr[:5] = 2 * np.arange(5, dtype=np.int16)[::-1]
dt_factorial(arr), np.prod(factorial(arr))

(1393459200, 1393459200.0)

In [25]:
%%timeit
_dt_factorial(arr)

164 µs ± 5.82 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [26]:
%%timeit
np.prod(factorial(arr))

7.85 µs ± 447 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [27]:
%%timeit
dt_factorial(arr)

274 ns ± 3.12 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [28]:
%%timeit
np.sum(arr)

3.64 µs ± 281 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [29]:
%%timeit
dt_sum(arr)

226 ns ± 34.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [30]:
sort_and_fill(np.arange(3*5, dtype=np.int16).reshape((3, 5)), 10)

array([[ 4,  3,  2,  1,  0,  0,  0,  0,  0,  0],
       [ 9,  8,  7,  6,  5,  0,  0,  0,  0,  0],
       [14, 13, 12, 11, 10,  0,  0,  0,  0,  0]], dtype=int16)

#### Compute $\eta$

In [31]:
[l for l in compute_etas(5, 3, 3, np.array((3, 2, 0), dtype=np.int16))]

[array([[0, 1],
        [3, 1]], dtype=int16),
 array([[0, 2],
        [3, 0]], dtype=int16),
 array([[1, 0],
        [2, 2]], dtype=int16),
 array([[1, 1],
        [2, 1]], dtype=int16),
 array([[1, 2],
        [2, 0]], dtype=int16),
 array([[2, 0],
        [1, 2]], dtype=int16),
 array([[2, 1],
        [1, 1]], dtype=int16),
 array([[2, 2],
        [1, 0]], dtype=int16),
 array([[3, 0],
        [0, 2]], dtype=int16),
 array([[3, 1],
        [0, 1]], dtype=int16)]

In [32]:
%%timeit
_compute_etas(7, 3, 5, np.array((4, 3, 0), dtype=np.int16))

1.41 ms ± 89.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [33]:
%%timeit
compute_etas(7, 3, 5, np.array((4, 3, 0), dtype=np.int16))

35.8 µs ± 1.33 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


#### Compute $\zeta$
For $\zeta := (\zeta^{(1)}, \ldots, \zeta^{(n)}) \in (\mathbb N^k)^n$, there are the following constraints:
- $\zeta^{(i)} = 0$ for $ i < j$
- $\vert \zeta^{(i)} \vert \leq \vert \zeta^{(i+1)} \vert$ for $ i < n$ <-- Constraint on row
- $\sum_{i=j}^{n} \vert \eta^{(i)} \vert \zeta^{(i)} = \alpha$ <-- Constraint on column

In [34]:
n = 5; k = 10; m = 3; j=2; alpha=np.array((3, 2, 0), dtype=np.int16)
etas = compute_etas(n, m, j, alpha)
zetas = compute_zetas(n, k, j, etas[0], alpha)
etas[0], zetas[0], zetas[0] * etas[0].sum(axis=1)[:, None]

/Users/yannicklimmer/Development/Projects/gen-bounds-transformer/src/Combinatorics.py:61: NumbaTypeSafetyWarning: unsafe cast from int64 to int16. Precision may be lost.
  cur_allowance[i] = dt_sum(cur_arrs[i][row - 1])


(array([[0, 1],
        [0, 1],
        [3, 0]], dtype=int16),
 array([[0, 1],
        [0, 1],
        [1, 0]], dtype=int16),
 array([[0, 1],
        [0, 1],
        [3, 0]]))

In [35]:
n = 5; k = 10; m = 3; j=2; alpha=np.array((3, 2, 0), dtype=np.int16)
etas = compute_etas(n, m, j, alpha)
zetas = compute_zetas(n, k, j, etas[2], alpha)
[etas[2]] + [l for l in zetas]

[array([[0, 1],
        [1, 1],
        [2, 0]], dtype=int16),
 array([[1, 0],
        [0, 1],
        [1, 0]], dtype=int16),
 array([[1, 0],
        [1, 0],
        [0, 1]], dtype=int16)]

In [36]:
zetas = compute_sorted_zetas(n, k, j, etas[2], alpha)
[l for l in zetas]

[array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int16),
 array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int16)]

### Cummulative $g$ bounds

In [8]:
from src.DerivativeBounds import _compute_cumulated_g_bounds_for_zeta, _compute_cumulated_g_bounds_for_eta, \
    _compute_cumulated_g_bounds_for_j, _compute_cumulated_g_bounds, compute_cumulated_g_bounds_for_zeta, compute_cumulated_g_bounds_for_eta, \
    compute_cumulated_g_bounds_for_j, compute_cumulated_g_bounds

In [18]:
n = 5
m = 10
k = 10
der_types = list(generate_derivative_types(n, k))
h_der_type = der_types[1]
f_der_type = generate_derivative_types(n, m)[1]
for i in range(1, n):
    der_types += list(generate_derivative_types(i, k))
g_dbounds = make_dbound_dict(der_types_to_hashes(der_types, n, k), np.ones(len(der_types)))
compute_cumulated_g_bounds(n, m, k, h_der_type, f_der_type, g_dbounds)

721.0

In [19]:
%%timeit
_compute_cumulated_g_bounds(n, m, k, h_der_type, f_der_type, g_dbounds)

11.8 ms ± 1.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%%timeit
compute_cumulated_g_bounds(n, m, k, h_der_type, f_der_type, g_dbounds)

10.7 ms ± 213 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
compute_cumulated_g_bounds_for_j.parallel_diagnostics()

 
 Parallel Accelerator Optimizing:  Function _compute_cumulated_g_bounds_for_j, 
/Users/yannicklimmer/Development/Projects/gen-bounds-
transformer/src/DerivativeBounds.py (63)  


Parallel loop listing for  Function _compute_cumulated_g_bounds_for_j, /Users/yannicklimmer/Development/Projects/gen-bounds-transformer/src/DerivativeBounds.py (63) 
---------------------------------------------------------------------|loop #ID
def _compute_cumulated_g_bounds_for_j(                               | 
        n: np.int16,                                                 | 
        m: np.int16,                                                 | 
        k: np.int16,                                                 | 
        j: np.int16,                                                 | 
        h_der_type: nb.int16[::1],                                   | 
        f_der_type: nb.int16[::1],                                   | 
        g_dbounds: DBoundDict,                                       |

## Compute Representations of $\beta$

In [22]:
binomial_coefficient(10, 2)

40

In [23]:
beta = np.zeros(1000)
beta[:4] = np.array((1, 1, 1, 0), dtype=np.int16)
number_of_representations(beta)

165668000

In [24]:
%%timeit
_number_of_representations(beta)

1.03 µs ± 38.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [25]:
%%timeit
number_of_representations(beta)

240 ns ± 1.85 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Compute Bound for $\alpha$

In [45]:
def compute_bound(
    n: np.int16,
    m: np.int16,
    k: np.int16,
    h_der_type: nb.int16[::1],
    f_der_types: List[nb.int16[::1]],
    f_dbounds: DBoundDict,
    g_der_types: List[nb.int16[::1]],
    g_dbounds: DBoundDict,
):
    result = 0
    
    for fi in range(len(f_der_types)):
        f_der_type = f_der_types[fi]
        f_dbound = f_dbounds[der_type_to_hash(f_der_type, n, m)]

        representations_for_der_type = number_of_representations(n, m, f_der_type)
        cummulated_g_bounds = compute_cumulated_g_bounds(n, m, k, h_der_type, f_der_type, g_dbounds)
        result += representations_for_der_type * f_dbound * cummulated_g_bounds
        
    return result